## Step 0: Import Python packages

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [ ]:
print('hello')

## Step 1: Download BigQuery data to our notebook


In [18]:
import os

from google.cloud import bigquery

# Find your project number 
project_number = '697349885677'

client = bigquery.Client(project=project_number)

In [19]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = client.query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [ ]:
df.describe()

In [4]:
df['is_male'].value_counts()


True     5150
False    4850
Name: is_male, dtype: int64

## Step 2: Prepare the dataset for training

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [9]:
## Step 3: Split the data into train test sets

In [10]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

2022-06-02 20:14:58.207542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-06-02 20:14:58.207602: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-02 20:14:58.207640: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (e0c16f336b4a): /proc/driver/nvidia/version does not exist
2022-06-02 20:14:58.207962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in 

In [12]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [13]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)


Epoch 1/10
209/209 [==============================] - 1s 2ms/step - loss: 1.7838 - mae: 1.0383 - mse: 1.7838 - val_loss: 1.9337 - val_mae: 1.1254 - val_mse: 1.9337
Epoch 2/10
209/209 [==============================] - 0s 2ms/step - loss: 1.5759 - mae: 0.9863 - mse: 1.5759 - val_loss: 1.1740 - val_mae: 0.8574 - val_mse: 1.1740
Epoch 3/10
209/209 [==============================] - 0s 2ms/step - loss: 1.4967 - mae: 0.9552 - mse: 1.4967 - val_loss: 1.1356 - val_mae: 0.8382 - val_mse: 1.1356
Epoch 4/10
209/209 [==============================] - 0s 2ms/step - loss: 1.4396 - mae: 0.9410 - mse: 1.4396 - val_loss: 1.1212 - val_mae: 0.8342 - val_mse: 1.1212
Epoch 5/10
209/209 [==============================] - 0s 2ms/step - loss: 1.4044 - mae: 0.9291 - mse: 1.4044 - val_loss: 1.8092 - val_mae: 1.0855 - val_mse: 1.8092
Epoch 6/10
209/209 [==============================] - 0s 2ms/step - loss: 1.3498 - mae: 0.9104 - mse: 1.3498 - val_loss: 1.7828 - val_mae: 1.0903 - val_mse: 1.7828
Epoch 7/10
209/2

In [14]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

1/1 [==============================] - 0s 81ms/step


In [15]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.0147777
Actual val:  6.0627122049999995

Predicted val:  6.588154
Actual val:  6.686620406459999

Predicted val:  7.3424263
Actual val:  8.2342654857

Predicted val:  7.481974
Actual val:  8.858173687159999

Predicted val:  7.1853013
Actual val:  9.57026679342

Predicted val:  7.223127
Actual val:  7.7933409617

Predicted val:  6.806429
Actual val:  9.1932763254

Predicted val:  6.962726
Actual val:  6.75055446244

Predicted val:  7.248742
Actual val:  8.6421206704

Predicted val:  6.767269
Actual val:  7.25100379718



In [ ]:
## Register the model

In [16]:
model